# 8. Resolve data issues

The analysis in section 8.2 is based on data retrieved from LexisNexis (LexisNexis Dossier).

In [1]:
sample_file = '../preprocessed_data/sample_2019-09-02.feather'
issues_to_address = '../preprocessed_data/issues_to_address_2019-09-02.pickle'
pipelines_2010_raw_file = '../data/pipelines_2010_2019-08-16.feather'

## Setup

In [2]:
import pandas as pd
import numpy as np
from datetime import date
from functools import partial

today = date.today().isoformat()

In [3]:
import wrds

db = wrds.Connection(wrds_username='juujian')

Loading library list...
Done


## Load data

In [4]:
import pickle

with open(issues_to_address, 'rb') as file:
    issues = pickle.load(file)
    
issues = (issue for issue in issues)

In [5]:
sample = pd.read_feather(sample_file)
sample.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,PERC_OFFSHORE,AVG_AGE,PARENT,INCIDENTS,SIGNIFICANT_INCIDENTS
383,12628,2011,EXXONMOBIL PIPELINE CO,hvl,66.00,0.00,0.00,0.00,80.00,56.50,73.00,15.00,7.00,12.50,0.00,0.0,51.147541,Exxon Mobil,0.0,0.0
649,22855,2016,"KOCH PIPELINE COMPANY, L.P.",crude,582.23,0.00,0.00,0.00,213.88,0.00,256.90,767.54,76.58,317.21,143.92,0.0,33.624911,Koch Industries,1.0,1.0
611,22610,2015,"MAGELLAN PIPELINE COMPANY, LP",crude,463.75,0.75,0.00,60.75,34.00,131.75,191.00,3.50,16.00,32.25,486.25,0.0,27.257980,Magellan Midstream Partners,5.0,2.0
1446,32296,2017,TARGA RESOURCES OPERATING LLC,hvl,150.15,8.71,26.98,4.33,59.79,166.11,147.69,30.69,28.77,4.24,11.43,0.0,50.695894,Targa Resources,0.0,0.0
1426,32258,2015,KINDER MORGAN COCHIN LLC,hvl,187.71,0.00,0.00,0.00,0.25,0.00,262.77,0.00,38.86,0.00,1.39,0.0,42.270419,Kinder Morgan,0.0,0.0


In [6]:
pipelines_2010_raw = pd.read_feather(pipelines_2010_raw_file)

## 8.1 Functions for analysis

In [7]:
from functools import partial

def find_info(OPERATOR_ID, info_col: str, title: str, df = pipelines_2010_raw, id_col = 'OPERATOR_ID', 
              year_col = 'REPORT_YEAR', fuzzy=False):
    from fuzzywuzzy import fuzz
    
    values = np.unique(df[df[id_col] == int(OPERATOR_ID)][info_col]).tolist()
    result = []
    for value in values:
        start_year = df[df[info_col] == value][year_col].min()
        end_year = df[df[info_col] == value][year_col].max()
        id_ = OPERATOR_ID
        result = result + [{title: value, 'start_year': start_year, 'end_year': end_year, 'id_': id_}]
        
    if fuzzy and len(result) == 2 and fuzz.ratio(result[0][title].lower(), result[1][title].lower()) >= 95:
            result = [result[0]]
            
    return(result)

find_address = partial(find_info, info_col='PARTA4STREET', title='address')
find_name = partial(find_info, info_col='PARTA2NAMEOFCOMP', title='name', fuzzy=True)
find_name('3445')

[{'name': 'DIXIE PIPELINE',
  'start_year': 2010,
  'end_year': 2010,
  'id_': '3445'},
 {'name': 'DIXIE PIPELINE COMPANY LLC',
  'start_year': 2011,
  'end_year': 2018,
  'id_': '3445'}]

In [8]:
def report_names(names: list):
    for name in names: 
        hist = find_name(name)
        if len(hist) > 1:
            print(f"\nOrganization {hist[0]['id_']} has changed its name:")
            for row in hist:
                print(f"    {row['id_']} was named {row['name']} from {row['start_year']} to {row['end_year']}.")
    
    if max([len(find_name(name)) for name in names]) < 2:
        print('No organizations were renamed at any time.')

report_names(['4907', '3445'])


Organization 3445 has changed its name:
    3445 was named DIXIE PIPELINE from 2010 to 2010.
    3445 was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.


In [9]:
def find_total_miles_year(OPERATOR_ID, year, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', 
                          id_col = 'OPERATOR_ID', miles_col='PARTBHCATOTAL'):
    observations = df.loc[(df[year_col] == year) & (df[id_col] == int(OPERATOR_ID))][miles_col]
    return observations.sum()

find_total_miles_year('4906', 2015)

2113.75

In [10]:
def find_total_miles(OPERATOR_ID, years, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', miles_col='PARTBHCATOTAL'):
    result = []
    for year in years:
        result.append([year, find_total_miles_year(OPERATOR_ID, year)])
    return(result)

find_total_miles('4906', range(2013, 2017))

[[2013, 2944.4], [2014, 2343.06], [2015, 2113.75], [2016, 2590.17]]

## 8.2 Iterate over issues and resolve them one at a time

We use the dictionary "company groups" to capture any companies we have to combine to form one observation. At every step we consult ownership_changes.txt to see if there are changed in ownership over time.

### Store results in

In [11]:
company_groups = pd.DataFrame()

def add_company_group(name: str, members: list, df = company_groups):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    df = df.append(new_group, ignore_index=True)
    return df

In [12]:
m_as = pd.DataFrame()

def add_m_a(name: str, members: list, df, start_year: str = None, end_year: str = None):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    
    if start_year:
        new_group['start_year'] = start_year
    if end_year:
        new_group['end_year'] = end_year
    
    df = df.append(new_group, sort=False, ignore_index=True)
    return df

In [13]:
spin_offs = []

### 8.2.1 Enterprise Products Operating

In [14]:
print(next(issues))


ENTERPRISE PRODUCTS OPERATING LLC (OPERATOR_ID 31618) has the same parent company as:

	DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445)

	ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829)

	TRI-STATES NGL PIPELINE LLC (OPERATOR_ID 31270)



### Gather infor_asation

In [15]:
report_names(['31618', '3445', '30829', '31270'])


Organization 3445 has changed its name:
    3445 was named DIXIE PIPELINE from 2010 to 2010.
    3445 was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.


### Decision

As per ownership_changes.txt, enterprise products partners aqcuired a majority stake in Dixie Pipelines in 2005. The others are wholly-owned subsidies.

In [16]:
company_groups = add_company_group('Enterprise Products (Group)', ['31618', '3445', '30829', '31270'], company_groups)
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,3445,Enterprise Products (Group)
2,30829,Enterprise Products (Group)
3,31270,Enterprise Products (Group)


### 8.2.2 Oneok NGL

In [17]:
print(next(issues))


ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) has changed its name:

	Was named ONEOK NGL PIPELINE LP from 2010 to 2011.

	Was named ONEOK NGL PIPELINE, LLC from 2012 to 2018.



No action necessary.

### 8.2.3 Magellan

In [18]:
print(next(issues))


MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) has the same parent company as:

	MAGELLAN AMMONIA PIPELINE, L.P. (OPERATOR_ID 12105)

	MAGELLAN PIPELINES HOLDINGS, LP (OPERATOR_ID 31579)

	MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P. (OPERATOR_ID 39504)

MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)



In [19]:
report_names(['22610', '12105', '31579', '39504'])

No organizations were renamed at any time.


LexisNexis yielded no reports of ownership changed for any subsidiaries (some assets changed hands, but that case is appropriately covered by the FERC data).

### Decision

In [20]:
company_groups = add_company_group('Magellan (Group)', ['22610', '12105', '31579', '39504'], company_groups)
company_groups.tail()

,members,name
3,31270,Enterprise Products (Group)
4,22610,Magellan (Group)
5,12105,Magellan (Group)
6,31579,Magellan (Group)
7,39504,Magellan (Group)


### 8.2.4 Phillips 66

In [21]:
print(next(issues))


PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has changed its name:

	Was named CONOCOPHILLIPS from 2010 to 2010.

	Was named PHILLIPS 66 PIPELINE LLC from 2011 to 2018.

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has the same parent company as:

	PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485)

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) may have a namesake or namesakes:

	CONOCOPHILLIPS COMPANY (OPERATOR_ID 15485)

	PHILLIPS 66 COMPANY (OPERATOR_ID 15485)



Downstream business was spun off in 2011 as Phillips 66.

In [22]:
report_names(['31684', '15485'])


Organization 31684 has changed its name:
    31684 was named CONOCOPHILLIPS from 2010 to 2010.
    31684 was named PHILLIPS 66 PIPELINE LLC from 2011 to 2018.

Organization 15485 has changed its name:
    15485 was named CONOCOPHILLIPS COMPANY from 2010 to 2010.
    15485 was named PHILLIPS 66 COMPANY from 2011 to 2011.
    15485 was named PHILLIPS 66 COMPANY - SWEENY REFINERY from 2012 to 2018.


Sweeny refinery is not an independent organizations. We will reconsolidate the two organizations.

### Decision

In [23]:
company_groups = add_company_group('Phillips 66 (Group)', ['15485', '31684'], company_groups)
company_groups.tail()

,members,name
5,12105,Magellan (Group)
6,31579,Magellan (Group)
7,39504,Magellan (Group)
8,15485,Phillips 66 (Group)
9,31684,Phillips 66 (Group)


### 8.2.5 Buckeye

In [24]:
print(next(issues))


BUCKEYE PARTNERS, LP (OPERATOR_ID 1845) has the same parent company as:

	BUCKEYE DEVELOPMENT & LOGISTICS, LLC (OPERATOR_ID 31371)



In [25]:
report_names(['1845', '31371'])

No organizations were renamed at any time.


### Decision

LexisNexis does not indicate that Buckeye Development & Logistics is an outside acquisition by Buckeye Partners. We consolidate the two organizations into one.

In [26]:
company_groups = add_company_group('Buckeye (Group)', ['1845', '31371'], company_groups)
company_groups.tail()

,members,name
7,39504,Magellan (Group)
8,15485,Phillips 66 (Group)
9,31684,Phillips 66 (Group)
10,1845,Buckeye (Group)
11,31371,Buckeye (Group)


### 8.2.5 Sunoco

In [27]:
print(next(issues))


SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) has the same parent company as:

	MID - VALLEY PIPELINE CO (OPERATOR_ID 12470)

	WEST TEXAS GULF PIPELINE CO (OPERATOR_ID 22442)

	ENERGY TRANSFER COMPANY (OPERATOR_ID 32099)

	INLAND CORPORATION (OPERATOR_ID 32683)

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)



In [28]:
report_names(['18718', '12470', '22442', '32099', '32683', '39205', '39596'])

No organizations were renamed at any time.


According to our .txt file on ownership changes, Sunoco 

    - acquired a majority stake in Inland Corp (32683) on May 17, 2011
    
    - acquired a majority stake in West Texas Gulf Pipeline (22442) on Aug 18, 2010
    
    - merged with Energy Transfer (32099) on Oct 5, 2012

In [29]:
company_groups = add_company_group('Sunoco (Group)', ['18718', '12470', '39205', '39596'], company_groups)
company_groups.tail()

,members,name
11,31371,Buckeye (Group)
12,18718,Sunoco (Group)
13,12470,Sunoco (Group)
14,39205,Sunoco (Group)
15,39596,Sunoco (Group)


In [30]:
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', '32683', '22442'], m_as, start_year = '2011')
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', '32099'], m_as, start_year = '2013')

m_as

,members,name,start_year
0,Sunoco (Group),Sunoco (Group),2011
1,32683,Sunoco (Group),2011
2,22442,Sunoco (Group),2011
3,Sunoco (Group),Sunoco (Group),2013
4,32099,Sunoco (Group),2013


### 8.2.6 Flint Hills

In [31]:
print(next(issues))


FLINT HILLS RESOURCES, LC (OPERATOR_ID 22855) has changed its name:

	Was named FLINT HILLS RESOURCES, LC from 2017 to 2018.

	Was named KOCH PIPELINE COMPANY, L.P. from 2010 to 2016.



In [32]:
find_total_miles('22855', range(2015, 2019))

[[2015, 1885.19], [2016, 1674.88], [2017, 1711.48], [2018, 1750.7199999999998]]

The restructuring or sell-off does not seem to affect the organizations pipeline assets. We will ignore the rebranding efforts.

### 8.2.7 NuStar

In [33]:
print(next(issues))


NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) has the same parent company as:

	NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454)

	NUSTAR PERMIAN TRANSPORTATION AND STORAGE, LLC (OPERATOR_ID 39348)



In [34]:
report_names('10012')

No organizations were renamed at any time.


LexisNexis Doessier does not list any M&As for the two subsidiaries. They just seem to be two separate subsidiaries, that we can group together.

In [35]:
company_groups = add_company_group('NuStar', ['10012', '31454'], company_groups)
company_groups.tail()

,members,name
13,12470,Sunoco (Group)
14,39205,Sunoco (Group)
15,39596,Sunoco (Group)
16,10012,NuStar
17,31454,NuStar


### 8.2.8 Kinder Morgan

In [36]:
print(next(issues))


PLANTATION PIPE LINE CO (OPERATOR_ID 15674) has the same parent company as:

	CENTRAL FLORIDA PIPELINE CORP (OPERATOR_ID 2190)

	SFPP, LP (OPERATOR_ID 18092)

	CALNEV PIPELINE CO (OPERATOR_ID 26125)

	KINDER MORGAN CO2 CO. LP (OPERATOR_ID 31555)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)

	COPANO NGL SERVICES (MARKHAM), LLC (OPERATOR_ID 32541)

	HILAND CRUDE, LLC (OPERATOR_ID 32619)

	KINDER MORGAN CRUDE AND CONDENSATE LLC (OPERATOR_ID 32678)

	DOUBLE H (OPERATOR_ID 39440)



In [37]:
report_names(['15674', '2190', '18092', '26125', '31555', '32258', '32541', '32619', '32678', '39440'])

No organizations were renamed at any time.


According to our .txt file on ownership changes, Kinder Morgan:

    - has held a majority share in Plantation Pipe Line (15674) since June 16, 1999
    
    - sold off Cochin (32258) to Pembina on Aug 21, 2019

In [38]:
company_groups = add_company_group('Kinder Morgan (Group)', 
                                   ['15674', '2190', '18092', '26125', '31555', '32258', '32541', '32619', '32678', '39440'], 
                                   company_groups)
company_groups.tail()

,members,name
23,32258,Kinder Morgan (Group)
24,32541,Kinder Morgan (Group)
25,32619,Kinder Morgan (Group)
26,32678,Kinder Morgan (Group)
27,39440,Kinder Morgan (Group)


### 8.2.9 Enbridge

In [39]:
print(next(issues))


ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) has the same parent company as:

	ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448)

	EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720)

	ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947)

	CCPS TRANSPORTATION, LLC (OPERATOR_ID 32080)

	ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502)

ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) may have a namesake or namesakes:

	ENBRIDGE PIPELINES (NORTH DAKOTA) LLC (OPERATOR_ID 15774)



In [40]:
report_names(['11169',  '31448', '31720', '31947', '32080', '32502', '15774'])


Organization 31720 has changed its name:
    31720 was named EXPRESS HOLDINGS (USA), LLC from 2013 to 2018.
    31720 was named KINDER MORGAN PIPELINES (USA) INC from 2010 to 2012.

Organization 15774 has changed its name:
    15774 was named ENBRIDGE PIPELINES (NORTH DAKOTA) LLC from 2010 to 2012.
    15774 was named NORTH DAKOTA PIPELINE COMPANY LLC from 2013 to 2018.


### Express Holdings

In [41]:
find_total_miles('31720', range(2010, 2015))

[[2010, 196.0], [2011, 196.0], [2012, 196.0], [2013, 219.3], [2014, 212.4]]

Spectra Energy bought Express Pipeline (31720) in 2012 from a number of stakeholders, including Kinder Morgan (Deal Number 741207). Enbridge merged with Spectra Energy on Feb 27, 2017 (LexisNexis Dossier Deal Number 885812).

In [42]:
company_groups = add_company_group('Enbridge (Group)', 
                                   ['11169',  '31448', '31947', '32080', '32502', '15774'], company_groups)
company_groups.tail()

,members,name
29,31448,Enbridge (Group)
30,31947,Enbridge (Group)
31,32080,Enbridge (Group)
32,32502,Enbridge (Group)
33,15774,Enbridge (Group)


### 9.2.10 Marathon

In [43]:
print(next(issues))


MARATHON PIPE LINE LLC (OPERATOR_ID 32147) has the same parent company as:

	NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774)

	WOLVERINE PIPELINE CO (OPERATOR_ID 22830)

	MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026)

	TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570)

	WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574)

	TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933)

	TESORO SOCAL PIPELINE COMPANY LLC (OPERATOR_ID 39013)

	TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029)

	ILLINOIS EXTENSION PIPELINE COMPANY, L.L.C. (OPERATOR_ID 39347)



In [44]:
report_names(['32147', '15774', '22830', '26026', '31570', '31574', '38933', '39013', '39029', '39347'])


Organization 15774 has changed its name:
    15774 was named ENBRIDGE PIPELINES (NORTH DAKOTA) LLC from 2010 to 2012.
    15774 was named NORTH DAKOTA PIPELINE COMPANY LLC from 2013 to 2018.

Organization 26026 has changed its name:
    26026 was named MARKWEST ENERGY APPALACHIA, LLC from 2010 to 2010.
    26026 was named MARKWEST RANGER PIPELINE COMPANY, L.L.C. from 2012 to 2018.
    26026 was named MARKWEST RANGER PIPELINE COMPANY, LLC from 2011 to 2011.

Organization 31570 has changed its name:
    31570 was named TESORO - HIGH PLAINS PIPELINE COMPANY from 2010 to 2011.
    31570 was named TESORO HIGH PLAINS PIPELINE COMPANY LLC from 2012 to 2017.

Organization 31574 has changed its name:
    31574 was named WESTERN REFINING LOGISTICS, LP from 2018 to 2018.
    31574 was named WESTERN REFINING PIPELINE COMPANY from 2010 to 2012.
    31574 was named WESTERN REFINING PIPELINE, LLC from 2013 to 2017.

Organization 38933 has changed its name:
    38933 was named TESORO LOGISTICS OPERAT

As we have touched on above, the North Dakota pipeline's owner was named "Enbridge Pipelines", but it was actually owned by a consortium of different actors.

LexisNexis indicates that Tesoro was renamed to Andeavor, and acquired by Marathon on Oct 1, 2018 (LexisNexis Dossier Deal Number 3052392). We will group Wolverine with Marathon, and also combine the two Tesoros into one observation. For 2019, we would subsume Tesoro under Marathon.

In [45]:
# Marathon and Wolverine
company_groups = add_company_group('Marathon (Group)', 
                                   ['32147', '22830', '26026', '31574', '39347'], company_groups)

# Tesoro
company_groups = add_company_group('Tesoro (Group)', ['31570', '38933', '39013', '39029'], company_groups)
company_groups.tail()

,members,name
38,39347,Marathon (Group)
39,31570,Tesoro (Group)
40,38933,Tesoro (Group)
41,39013,Tesoro (Group)
42,39029,Tesoro (Group)


### 8.2.11 ExxonMobil

In [46]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) has the same parent company as:

	EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624)

	EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628)

	EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634)

	EXXONMOBIL OIL CORP - WEST COAST (OPERATOR_ID 26134)



In [47]:
report_names(['4906', '12624', '12628', '12634', '26134'])


Organization 12624 has changed its name:
    12624 was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.
    12624 was named MOBIL CORP from 2010 to 2017.

Organization 12628 has changed its name:
    12628 was named EXXONMOBIL PIPELINE CO from 2010 to 2018.
    12628 was named MOBIL  PIPE  LINE COMPANY from 2012 to 2018.
    12628 was named MOBIL PIPELINE CO from 2010 to 2010.

Organization 12634 has changed its name:
    12634 was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.
    12634 was named MOBIL CHEMICAL CO from 2010 to 2017.


All are wholly owned subsidiaries of exxonmobil. LexisNexis indicates no M&A activities. We combine them into one observation.

In [48]:
company_groups = add_company_group('ExxonMobil (Group)', ['4906', '12624', '12628', '12634', '26134'], company_groups)
company_groups.tail()

,members,name
43,4906,ExxonMobil (Group)
44,12624,ExxonMobil (Group)
45,12628,ExxonMobil (Group)
46,12634,ExxonMobil (Group)
47,26134,ExxonMobil (Group)


### 8.2.12 ExxonMobil

In [49]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) has changed its name:

	Was named EXXONMOBIL PIPELINE CO from 2010 to 2018.

	Was named MOBIL  PIPE  LINE COMPANY from 2012 to 2018.

	Was named MOBIL PIPELINE CO from 2010 to 2010.

EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) may have a namesake or namesakes:

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL CHEMICAL CO (OPERATOR_ID 12634)



Already addressed above.

### 8.2.13 NuStar

In [50]:
print(next(issues))


PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596) may have a namesake or namesakes:

	NUSTAR PERMIAN TRANSPORTATION AND STORAGE, LLC (OPERATOR_ID 39348)



Already addressed above.

### 8.2.14 Tesoro

In [51]:
print(next(issues))


TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA from 2012 to 2017.

	Was named TESORO LOGISTICS OPERATIONS, LLC from 2018 to 2018.

TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)



Already addressed above.

### 8.2.15 Plains Pipeline

In [52]:
print(next(issues))


ROCKY MOUNTAIN PIPELINE SYSTEM, LLC (OPERATOR_ID 31666) has the same parent company as:

	PLAINS PIPELINE, L.P. (OPERATOR_ID 300)

	PLAINS MARKETING, L.P. (OPERATOR_ID 26085)



In [53]:
report_names(['31666', '300', '26085'])

No organizations were renamed at any time.


LexisNexis Doassiers indicates no M&As for the subsidiaries, Plains Marketing and Rocky Mountain Pipeline System.

In [54]:
company_groups = add_company_group('Plains Pipeline (Group)', ['300', '31666', '26085'], company_groups)
company_groups.tail()

,members,name
46,12634,ExxonMobil (Group)
47,26134,ExxonMobil (Group)
48,300,Plains Pipeline (Group)
49,31666,Plains Pipeline (Group)
50,26085,Plains Pipeline (Group)


### 8.2.16 Dixie Pipeline

In [55]:
print(next(issues))


DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445) has changed its name:

	Was named DIXIE PIPELINE from 2010 to 2010.

	Was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.



No action necessary.

### 8.2.17 Kinder Morgan

In [56]:
print(next(issues))


KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258) may have a namesake or namesakes:

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)



Already addressed above.

### 8.2.18 BP

In [57]:
print(next(issues))


BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) has the same parent company as:

	BP OIL PIPELINE CO (OPERATOR_ID 18386)

	OLYMPIC PIPE LINE COMPANY (OPERATOR_ID 30781)

	BP WEST COAST PRODUCTS L.L.C. (OPERATOR_ID 31610)



In [58]:
report_names(['31189', '18386', '30781', '31610'])

No organizations were renamed at any time.


LexisNexis wrongly reports that BP is the parent of Olympic Pipe Line Company (30781). On Feb 1, 2006 Enbridge acquired a majority stake (LexisNexis Doassier Deal Number 358159).

In [59]:
company_groups = add_company_group('BP (Group)', ['31189', '18386', '31610'], company_groups)
company_groups = add_company_group('Enbridge (Group)', ['30781'], company_groups)

company_groups.tail()

,members,name
50,26085,Plains Pipeline (Group)
51,31189,BP (Group)
52,18386,BP (Group)
53,31610,BP (Group)
54,30781,Enbridge (Group)


### 8.2.19 Holly

In [60]:
print(next(issues))


HOLLY ENERGY PARTNERS - OPERATING, L.P. (OPERATOR_ID 32011) has the same parent company as:

	UNEV PIPELINE, LLC (OPERATOR_ID 32493)

	FRONTIER PIPELINE CO (OPERATOR_ID 5656)



In [61]:
report_names(['32011', '32493', '5656'])

No organizations were renamed at any time.


In [62]:
company_groups = add_company_group('HollyFrontier (Group)', ['32011', '32493', '5656'], company_groups)
company_groups.tail()

,members,name
53,31610,BP (Group)
54,30781,Enbridge (Group)
55,32011,HollyFrontier (Group)
56,32493,HollyFrontier (Group)
57,5656,HollyFrontier (Group)


### 8.2.20 Tesoro

In [63]:
print(next(issues))


TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



### 8.2.21 Williams Field Services 

In [64]:
print(next(issues))


WILLIAMS FIELD SERVICES (OPERATOR_ID 30826) has the same parent company as:

	WILLIAMS FIELD SERVICES - GULF COAST COMPANY, LP (OPERATOR_ID 994)

	WILLIAMS OLEFINS FEEDSTOCK PIPELINES, LLC (OPERATOR_ID 32614)



In [65]:
report_names(['30826', '994', '32614'])

No organizations were renamed at any time.


In [66]:
company_groups = add_company_group('Williams Field Services (Group)', ['30826', '994', '32614'], company_groups)
company_groups.tail()

,members,name
56,32493,HollyFrontier (Group)
57,5656,HollyFrontier (Group)
58,30826,Williams Field Services (Group)
59,994,Williams Field Services (Group)
60,32614,Williams Field Services (Group)


### 8.2.22 Express Holdings

In [67]:
print(next(issues))


EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) has changed its name:

	Was named EXPRESS HOLDINGS (USA), LLC from 2013 to 2018.

	Was named KINDER MORGAN PIPELINES (USA) INC from 2010 to 2012.

EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) may have a namesake or namesakes:

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN CO2 CO. LP (OPERATOR_ID 31555)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)

	KINDER MORGAN CRUDE AND CONDENSATE LLC (OPERATOR_ID 32678)



Kinder Morgan already adressed above (8.2.10). No new relevant namesakes.

### 8.2.23 Gensis Pipeline

In [68]:
print(next(issues))


GENESIS PIPELINE USA, L.P. (OPERATOR_ID 31045) has the same parent company as:

	GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 32407)



In [69]:
company_groups = add_company_group('Genesis Pipeline (Group)', ['31045', '32407'], company_groups)
company_groups.tail()

,members,name
58,30826,Williams Field Services (Group)
59,994,Williams Field Services (Group)
60,32614,Williams Field Services (Group)
61,31045,Genesis Pipeline (Group)
62,32407,Genesis Pipeline (Group)


### 8.2.24 BKEP

In [70]:
print(next(issues))


BKEP PIPELINE, LLC (OPERATOR_ID 32551) has the same parent company as:

	BKEP CRUDE, LLC (OPERATOR_ID 32481)



In [71]:
company_groups = add_company_group('BKEP (Group)', ['32551', '32481'], company_groups)
company_groups.tail()

,members,name
60,32614,Williams Field Services (Group)
61,31045,Genesis Pipeline (Group)
62,32407,Genesis Pipeline (Group)
63,32551,BKEP (Group)
64,32481,BKEP (Group)


### 8.2.25 Phillips 66

In [72]:
print(next(issues))


PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485) has changed its name:

	Was named CONOCOPHILLIPS COMPANY from 2010 to 2010.

	Was named PHILLIPS 66 COMPANY from 2011 to 2011.

	Was named PHILLIPS 66 COMPANY - SWEENY REFINERY from 2012 to 2018.

PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485) may have a namesake or namesakes:

	CONOCOPHILLIPS (OPERATOR_ID 31684)



Covered above.

### 8.2.26 Plains Marketing

In [73]:
print(next(issues))


PLAINS MARKETING, L.P. (OPERATOR_ID 26085) may have a namesake or namesakes:

	TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570)

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	GREAT SALT PLAINS PIPELINE LLC (OPERATOR_ID 39401)



Covered above.

### 8.2.27 Targa Resources

In [74]:
print(next(issues))


TARGA RESOURCES OPERATING LLC (OPERATOR_ID 32296) has the same parent company as:

	TARGA NGL PIPE LINE CO (OPERATOR_ID 30626)



In [75]:
company_groups = add_company_group('Targa (Group)', ['32296', '30626'], company_groups)
company_groups.tail()

,members,name
62,32407,Genesis Pipeline (Group)
63,32551,BKEP (Group)
64,32481,BKEP (Group)
65,32296,Targa (Group)
66,30626,Targa (Group)


### 8.2.28 Dow

In [76]:
print(next(issues))


SEADRIFT PIPELINE CORP (OPERATOR_ID 26086) has the same parent company as:

	DOW PIPELINE CO (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY (OPERATOR_ID 30959)



In [77]:
company_groups = add_company_group('Dow (Group)', ['26086', '3527', '30959'], company_groups)
company_groups.tail()

,members,name
65,32296,Targa (Group)
66,30626,Targa (Group)
67,26086,Dow (Group)
68,3527,Dow (Group)
69,30959,Dow (Group)


### 8.2.29 CHS

In [78]:
print(next(issues))


CENEX PIPELINE LLC (OPERATOR_ID 2170) has the same parent company as:

	JAYHAWK PIPELINE LLC (OPERATOR_ID 9175)

	CHS MCPHERSON REFINERY INC. (OPERATOR_ID 26065)

	FRONT RANGE PIPELINE, LLC. (OPERATOR_ID 32283)



In [79]:
company_groups = add_company_group('CHS (Group)', ['2170', '9175', '26065', '32283'], company_groups)
company_groups.tail()

,members,name
69,30959,Dow (Group)
70,2170,CHS (Group)
71,9175,CHS (Group)
72,26065,CHS (Group)
73,32283,CHS (Group)


### 8.2.30 Harvest Midstream

In [80]:
print(next(issues))


HARVEST MIDSTREAM COMPANY (OPERATOR_ID 30782) has changed its name:

	Was named HARVEST MIDSTREAM COMPANY from 2018 to 2018.

	Was named HARVEST PIPELINE COMPANY from 2010 to 2017.



Not a concern.

### 8.2.30 Countrymark

In [81]:
print(next(issues))


COUNTRYMARK REFINING AND LOGISTICS, LLC (OPERATOR_ID 26049) has changed its name:

	Was named COUNTRYMARK COOPERATIVE, LLP from 2010 to 2010.

	Was named COUNTRYMARK REFINING AND LOGISTICS, LLC from 2011 to 2018.



Not a concern.

### 8.2.31 Magellan

In [82]:
print(next(issues))


MAGELLAN AMMONIA PIPELINE, L.P. (OPERATOR_ID 12105) may have a namesake or namesakes:

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)



Already covered in 8.2.3.

### 8.2.32 Occidental Petroleum

In [83]:
print(next(issues))


CENTURION PIPELINE L.P. (OPERATOR_ID 31888) has the same parent company as:

	BRAVO PIPELINE COMPANY (OPERATOR_ID 30657)



In [84]:
company_groups = add_company_group('Occidental Petroleum (Group)', ['31888', '30657'], company_groups)
company_groups.tail()

,members,name
71,9175,CHS (Group)
72,26065,CHS (Group)
73,32283,CHS (Group)
74,31888,Occidental Petroleum (Group)
75,30657,Occidental Petroleum (Group)


### 8.2.33 Valero

In [85]:
print(next(issues))


VALERO PARTNERS OPERATING CO. LLC (OPERATOR_ID 39105) has the same parent company as:

	VALERO TERMINALING AND DISTRIBUTION COMPANY (OPERATOR_ID 4430)

	PARKWAY PIPELINE LLC (OPERATOR_ID 32679)



In [86]:
report_names(['39105', '4430', '32679'])

No organizations were renamed at any time.


According to our notes on ownership changes, Valero acquired all of Parkway pipeline (32679) on July 12, 2016.

In [87]:
company_groups = add_company_group('Valero Energy (Group)', ['39105', '4430'], company_groups)
company_groups.tail()

,members,name
73,32283,CHS (Group)
74,31888,Occidental Petroleum (Group)
75,30657,Occidental Petroleum (Group)
76,39105,Valero Energy (Group)
77,4430,Valero Energy (Group)


In [88]:
m_as = add_m_a('Valero Energy (Group)', ['Valero Energy (Group)', '32679'], m_as, start_year = '2017')


### 8.2.34 Magellan

In [89]:
print(next(issues))


MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P. (OPERATOR_ID 39504) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)



Already handled above.

### 8.2.35 Targa

In [90]:
print(next(issues))


TARGA NGL PIPE LINE CO (OPERATOR_ID 30626) may have a namesake or namesakes:

	TARGA RESOURCES OPERATING LP (OPERATOR_ID 32296)



Already handled above.

### 8.2.36 SemGroup

In [91]:
print(next(issues))


WHITE CLIFFS PIPELINE, LLC (OPERATOR_ID 32288) has the same parent company as:

	ROSE ROCK MIDSTREAM L.P. (OPERATOR_ID 31476)



See ownership_changes.txt in folder input.

In [92]:
m_as = add_m_a('SemGroup (Group)', ['32288', '31476'], m_as, end_year = '2011')
m_as.tail()

,members,name,start_year,end_year
4,32099,Sunoco (Group),2013,NaN
5,Valero Energy (Group),Valero Energy (Group),2017,NaN
6,32679,Valero Energy (Group),2017,NaN
7,32288,SemGroup (Group),NaN,2011
8,31476,SemGroup (Group),NaN,2011


### 8.2.37 Petrologistics

In [93]:
print(next(issues))


PETROLOGISTICS OLEFINS, LLC (OPERATOR_ID 20160) has changed its name:

	Was named PETROLOGISTICS OLEFINS, LLC from 2011 to 2012.

	Was named PL Midstream LLC from 2010 to 2010.



### 8.2.37 Enlink

In [94]:
print(next(issues))


ENLINK LBU-GAS&LIQUIDS (OPERATOR_ID 32005) has the same parent company as:

	ENLINK NGL PIPELINE, LP (OPERATOR_ID 32107)



In [95]:
company_groups = add_company_group('Enlink (Group)', ['32005', '32107'], company_groups)
company_groups.tail()

,members,name
75,30657,Occidental Petroleum (Group)
76,39105,Valero Energy (Group)
77,4430,Valero Energy (Group)
78,32005,Enlink (Group)
79,32107,Enlink (Group)


### 3.2.38 ExxonMobil

In [96]:
print(next(issues))


EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634) has changed its name:

	Was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.

	Was named MOBIL CHEMICAL CO from 2010 to 2017.

EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634) may have a namesake or namesakes:

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL  PIPE  LINE COMPANY (OPERATOR_ID 12628)

	MOBIL PIPELINE CO (OPERATOR_ID 12628)



Already handled above.

### 8.2.39 Rose Rock

In [97]:
print(next(issues))


ROSE ROCK MIDSTREAM L.P. (OPERATOR_ID 31476) has changed its name:

	Was named ROSE ROCK MIDSTREAM L.P. from 2012 to 2018.

	Was named Rose Rock Midstream L.P. from 2011 to 2011.

	Was named SEMGROUP LP from 2010 to 2010.



Already handled above.

### 8.2.40 Eastman Chemical

In [98]:
print(next(issues))


MUSTANG  PIPE  LINE  LLC (OPERATOR_ID 31166) has changed its name:

	Was named MUSTANG  PIPE  LINE  LLC from 2012 to 2018.

	Was named MUSTANG PIPE LINE PARTNERS from 2010 to 2011.

MUSTANG  PIPE  LINE  LLC (OPERATOR_ID 31166) has the same parent company as:

	TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO (OPERATOR_ID 26103)



In [99]:
company_groups = add_company_group('Eastman Chemical (Group)', ['31166', '26103'], company_groups)
company_groups.tail()

,members,name
77,4430,Valero Energy (Group)
78,32005,Enlink (Group)
79,32107,Enlink (Group)
80,31166,Eastman Chemical (Group)
81,26103,Eastman Chemical (Group)


### 8.2.41 ExxonMobil

In [100]:
print(next(issues))


EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624) has changed its name:

	Was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.

	Was named MOBIL CORP from 2010 to 2017.

EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624) may have a namesake or namesakes:

	MOBIL  PIPE  LINE COMPANY (OPERATOR_ID 12628)

	MOBIL PIPELINE CO (OPERATOR_ID 12628)

	MOBIL CHEMICAL CO (OPERATOR_ID 12634)



Already handled above.

### 8.2.42 Enbridge

In [101]:
print(next(issues))


NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774) has changed its name:

	Was named ENBRIDGE PIPELINES (NORTH DAKOTA) LLC from 2010 to 2012.

	Was named NORTH DAKOTA PIPELINE COMPANY LLC from 2013 to 2018.

NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774) may have a namesake or namesakes:

	ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169)

	ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448)

	ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947)

	ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502)



Already handled above.

### 8.2.43 Markwest

In [102]:
print(next(issues))


MARKWEST LIBERTY MIDSTREAM & RESOURCES, L.L.C. (OPERATOR_ID 32412) may have a namesake or namesakes:

	MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026)

	MARKWEST RANGER PIPELINE COMPANY, LLC (OPERATOR_ID 26026)

	MARKWEST ENERGY APPALACHIA, LLC (OPERATOR_ID 26026)



Already handled above.

### 8.2.44 Dominion

In [103]:
print(next(issues))


DOMINION ENERGY TRANSMISSION, INC. (OPERATOR_ID 2714) has changed its name:

	Was named DOMINION ENERGY TRANSMISSION, INC. from 2017 to 2018.

	Was named DOMINION TRANSMISSION, INC from 2010 to 2016.



Not an issue we need to address.

### 8.2.45 Western Refining

In [104]:
print(next(issues))


WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574) has changed its name:

	Was named WESTERN REFINING LOGISTICS, LP from 2018 to 2018.

	Was named WESTERN REFINING PIPELINE COMPANY from 2010 to 2012.

	Was named WESTERN REFINING PIPELINE, LLC from 2013 to 2017.



Not an issue we need to address.

### 8.2.46 Delek

In [105]:
print(next(issues))


DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) has changed its name:

	Was named DELEK LOGISTICS OPERATING, LLC. from 2013 to 2018.

	Was named LION OIL TRADING & TRANSPORTATION, INC from 2010 to 2012.

DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) has the same parent company as:

	DELEK MARKETING AND SUPPLY, LP (OPERATOR_ID 15851)

	DELEK CRUDE LOGISTICS, LLC. (OPERATOR_ID 26061)

	PARAMOUNT PETROLEUM CORP (OPERATOR_ID 26136)

DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) may have a namesake or namesakes:

	DELEK PIPELINE TEXAS, INC (OPERATOR_ID 26061)



In [106]:
report_names(['11551', '15851', '26061', '26136'])


Organization 11551 has changed its name:
    11551 was named DELEK LOGISTICS OPERATING, LLC. from 2013 to 2018.
    11551 was named LION OIL TRADING & TRANSPORTATION, INC from 2010 to 2012.

Organization 26061 has changed its name:
    26061 was named DELEK CRUDE LOGISTICS, LLC. from 2013 to 2018.
    26061 was named DELEK PIPELINE TEXAS, INC from 2010 to 2012.


LexisNexis does not indicate that Delek bought Lion Oil.

In [107]:
company_groups = add_company_group('Delek (Group)', ['11551', '15851', '26061', '26136'], company_groups)
company_groups.tail()

,members,name
81,26103,Eastman Chemical (Group)
82,11551,Delek (Group)
83,15851,Delek (Group)
84,26061,Delek (Group)
85,26136,Delek (Group)


### 8.2.47 Suncor

In [108]:
print(next(issues))


PORTLAND PIPE LINE CORPORATION (OPERATOR_ID 15786) has changed its name:

	Was named PORTLAND PIPE LINE CORPORATION from 2012 to 2018.

	Was named PORTLAND PIPELINE CORP from 2010 to 2011.

PORTLAND PIPE LINE CORPORATION (OPERATOR_ID 15786) has the same parent company as:

	SUNCOR ENERGY (USA) PIPELINE CO. (OPERATOR_ID 31822)



In [109]:
company_groups = add_company_group('Suncor (Group)', ['15786', '31822'], company_groups)
company_groups.tail()

,members,name
83,15851,Delek (Group)
84,26061,Delek (Group)
85,26136,Delek (Group)
86,15786,Suncor (Group)
87,31822,Suncor (Group)


### 8.2.48 Crestwood Equity

In [110]:
print(next(issues))


ARROW PIPELINE (OPERATOR_ID 39083) has the same parent company as:

	CRESTWOOD MIDSTREAM PARTNERS LP (OPERATOR_ID 39368)



In [111]:
company_groups = add_company_group('Crestwood (Group)', ['39083', '39368'], company_groups)
company_groups.tail()

,members,name
85,26136,Delek (Group)
86,15786,Suncor (Group)
87,31822,Suncor (Group)
88,39083,Crestwood (Group)
89,39368,Crestwood (Group)


### 8.2.49 PBF Energy

In [112]:
print(next(issues))


TORRANCE VALLEY PIPELINE COMPANY LLC (OPERATOR_ID 39534) has the same parent company as:

	COLLINS PIPELINE CO (OPERATOR_ID 26120)

	TORRANCE PIPELINE COMPANY LLC (OPERATOR_ID 39535)



In [113]:
company_groups = add_company_group('PBF Energy (Group)', ['39534', '26120', '39535'], company_groups)
company_groups.tail()

,members,name
88,39083,Crestwood (Group)
89,39368,Crestwood (Group)
90,39534,PBF Energy (Group)
91,26120,PBF Energy (Group)
92,39535,PBF Energy (Group)


### 8.2.50 CHS McPherson

In [114]:
print(next(issues))


CHS MCPHERSON REFINERY INC. (OPERATOR_ID 26065) has changed its name:

	Was named CHS MCPHERSON REFINERY INC. from 2015 to 2018.

	Was named NATIONAL COOP REFINERY ASSCO from 2010 to 2014.



LexisNexis indicates not change of ownership.

### 8.2.51 Markwest

In [115]:
print(next(issues))


MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026) has changed its name:

	Was named MARKWEST ENERGY APPALACHIA, LLC from 2010 to 2010.

	Was named MARKWEST RANGER PIPELINE COMPANY, L.L.C. from 2012 to 2018.

	Was named MARKWEST RANGER PIPELINE COMPANY, LLC from 2011 to 2011.

MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026) may have a namesake or namesakes:

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, LLC (OPERATOR_ID 32412)

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, L.L.C. (OPERATOR_ID 32412)



Already addressed above.

### 8.2.52 Delek

In [116]:
print(next(issues))


TPC GROUP, LLC (OPERATOR_ID 19319) has changed its name:

	Was named TEXAS PETROCHEMICALS CORP from 2011 to 2011.

	Was named TPC GROUP, LLC from 2011 to 2018.

	Was named TPC Group LLC from 2010 to 2010.

TPC GROUP, LLC (OPERATOR_ID 19319) may have a namesake or namesakes:

	WEST TEXAS GULF PIPELINE CO (OPERATOR_ID 22442)

	DELEK PIPELINE TEXAS, INC (OPERATOR_ID 26061)

	TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO (OPERATOR_ID 26103)

	GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 32407)

	DENBURY GREEN PIPELINE-TEXAS, LLC (OPERATOR_ID 32543)

	ASCEND PERFORMANCE MATERIALS TEXAS INC (OPERATOR_ID 31517)



In [117]:
report_names(['19319', '22442', '26061', '26103', '32407', '32543', '31517'])


Organization 19319 has changed its name:
    19319 was named TEXAS PETROCHEMICALS CORP from 2011 to 2011.
    19319 was named TPC GROUP, LLC from 2011 to 2018.
    19319 was named TPC Group LLC from 2010 to 2010.

Organization 26061 has changed its name:
    26061 was named DELEK CRUDE LOGISTICS, LLC. from 2013 to 2018.
    26061 was named DELEK PIPELINE TEXAS, INC from 2010 to 2012.

Organization 31517 has changed its name:
    31517 was named ASCEND PERFORMANCE MATERIALS OPERATIONS LLC from 2012 to 2014.
    31517 was named ASCEND PERFORMANCE MATERIALS TEXAS INC from 2015 to 2016.
    31517 was named ASCEND PERFORMANCE MATERIALS, LLC from 2010 to 2011.


We have some false positives here, based on the term "Texas".

### 8.2.53 Ascend

In [118]:
print(next(issues))


ASCEND PERFORMANCE MATERIALS TEXAS INC (OPERATOR_ID 31517) has changed its name:

	Was named ASCEND PERFORMANCE MATERIALS OPERATIONS LLC from 2012 to 2014.

	Was named ASCEND PERFORMANCE MATERIALS TEXAS INC from 2015 to 2016.

	Was named ASCEND PERFORMANCE MATERIALS, LLC from 2010 to 2011.



Not a concern.

### 8.2.54 Tesoro

In [119]:
print(next(issues))


TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570) has changed its name:

	Was named TESORO - HIGH PLAINS PIPELINE COMPANY from 2010 to 2011.

	Was named TESORO HIGH PLAINS PIPELINE COMPANY LLC from 2012 to 2017.

TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570) may have a namesake or namesakes:

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



Not a concern.

### 8.2.55 LDH

In [120]:
print(next(issues))


LDH ENERGY PIPELINE L.P. (OPERATOR_ID 32035) has the same parent company as:

	LDH ENERGY MONT BELVIEU L.P. (OPERATOR_ID 32246)



In [121]:
report_names(['32035', '32246'])

No organizations were renamed at any time.


In [122]:
company_groups = add_company_group('LDH Energy (Group)', ['32035', '32246'], company_groups)
company_groups.tail()

,members,name
90,39534,PBF Energy (Group)
91,26120,PBF Energy (Group)
92,39535,PBF Energy (Group)
93,32035,LDH Energy (Group)
94,32246,LDH Energy (Group)


According to our notes, LDH was acquired by Energy Transfer Partners on March 8, 2017, which, at the time was already part of Sunoco.

In [123]:
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', 'LDH Energy (Group)'], m_as, start_year = '2017')
m_as.tail()

,members,name,start_year,end_year
6,32679,Valero Energy (Group),2017,NaN
7,32288,SemGroup (Group),NaN,2011
8,31476,SemGroup (Group),NaN,2011
9,Sunoco (Group),Sunoco (Group),2017,NaN
10,LDH Energy (Group),Sunoco (Group),2017,NaN


### 8.2.56 Enlink

In [124]:
print(next(issues))


ENLINK NGL PIPELINE, LP (OPERATOR_ID 32107) has changed its name:

	Was named CROSSTEX NGL PIPELINE, L.P. from 2010 to 2012.

	Was named ENLINK NGL PIPELINE, LP from 2013 to 2016.



LexisNexis does not indicate any M&A activity.

### 8.2.57 Magellan

In [125]:
print(next(issues))


MAGELLAN PIPELINES HOLDINGS, LP (OPERATOR_ID 31579) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)



No concern.

### 8.2.58 Delek

In [126]:
print(next(issues))


DELEK CRUDE LOGISTICS, LLC. (OPERATOR_ID 26061) has changed its name:

	Was named DELEK CRUDE LOGISTICS, LLC. from 2013 to 2018.

	Was named DELEK PIPELINE TEXAS, INC from 2010 to 2012.



Already covered above.

### 8.2.59 Glass Mountain

In [127]:
print(next(issues))


GLASS MOUNTAIN PIPELINE (OPERATOR_ID 39080) has the same parent company as:

	GLASS MOUNTAIN PIPELINE, LLC (OPERATOR_ID 39774)



In [128]:
find_total_miles('39080', range(2015, 2020))

[[2015, 72.5], [2016, 98.83], [2017, 0.0], [2018, 0.0], [2019, 0.0]]

In [129]:
find_total_miles('39774', range(2015, 2020))

[[2015, 0.0], [2016, 0.0], [2017, 98.83], [2018, 79.35], [2019, 0.0]]

In [130]:
report_names(['39080', '39774'])


Organization 39774 has changed its name:
    39774 was named GLASS MOUNTAIN PIPELINE, LLC from 2018 to 2018.
    39774 was named NAVIGATOR ENERGY SERVICES, LLC from 2017 to 2017.


LexisNexis does not indicate who the owner of Glass Mountain is, but shows that Navigator Energy was acquired by NuStar on May 4, 2017 (Deal Number 3004179). Since the two IDs obviously refer to the same organization, and this organization did not then continue reporting as part of NuStar, there is nothing we need to do other than group them together.

In [131]:
company_groups = add_company_group('Glass Mountain (Group)', ['39080', '39774'], company_groups)
company_groups.tail()

,members,name
92,39535,PBF Energy (Group)
93,32035,LDH Energy (Group)
94,32246,LDH Energy (Group)
95,39080,Glass Mountain (Group)
96,39774,Glass Mountain (Group)


Also, we know from our notes that Glass Mountain was part of SemGroup until Dec 2017, from at least 2015, but probably long before that.

In [132]:
m_as = add_m_a('SemGroup (Group)', ['SemGroup (Group)', 'Glass Mountain (Group)'], m_as, end_year = '2017')
m_as.tail()

,members,name,start_year,end_year
8,31476,SemGroup (Group),NaN,2011
9,Sunoco (Group),Sunoco (Group),2017,NaN
10,LDH Energy (Group),Sunoco (Group),2017,NaN
11,SemGroup (Group),SemGroup (Group),NaN,2017
12,Glass Mountain (Group),SemGroup (Group),NaN,2017


### 8.2.60 Glass Mountain

In [133]:
print(next(issues))


GLASS MOUNTAIN PIPELINE, LLC (OPERATOR_ID 39774) has changed its name:

	Was named GLASS MOUNTAIN PIPELINE, LLC from 2018 to 2018.

	Was named NAVIGATOR ENERGY SERVICES, LLC from 2017 to 2017.



Already addressed above.

### 8.2.61 Motiva

In [134]:
print(next(issues))


MOTIVA ENTERPRISE LLC (OPERATOR_ID 30777) has changed its name:

	Was named MOTIVA ENTERPRISE LLC from 2010 to 2018.

	Was named Motiva Enterprises LLC from 2010 to 2010.

	Was named Motiva Enterprises LLC Convent Refinery from 2010 to 2010.



Not an issue we need to address.

### 8.2.60 Kinder Morgan

In [135]:
print(next(issues))


KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041) may have a namesake or namesakes:

	KINDER MORGAN CO2 CO. LP (OPERATOR_ID 31555)

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)

	KINDER MORGAN CRUDE AND CONDENSATE LLC (OPERATOR_ID 32678)



Already addressed above.

In [136]:
print(next(issues))

StopIteration: 

## 8.3 Overview

In [137]:
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,3445,Enterprise Products (Group)
2,30829,Enterprise Products (Group)
3,31270,Enterprise Products (Group)
4,22610,Magellan (Group)
5,12105,Magellan (Group)
6,31579,Magellan (Group)
7,39504,Magellan (Group)
8,15485,Phillips 66 (Group)
9,31684,Phillips 66 (Group)


In [138]:
m_as

,members,name,start_year,end_year
0,Sunoco (Group),Sunoco (Group),2011,NaN
1,32683,Sunoco (Group),2011,NaN
2,22442,Sunoco (Group),2011,NaN
3,Sunoco (Group),Sunoco (Group),2013,NaN
4,32099,Sunoco (Group),2013,NaN
5,Valero Energy (Group),Valero Energy (Group),2017,NaN
6,32679,Valero Energy (Group),2017,NaN
7,32288,SemGroup (Group),NaN,2011
8,31476,SemGroup (Group),NaN,2011
9,Sunoco (Group),Sunoco (Group),2017,NaN


In [139]:
spin_offs

[]

## 8.4 Additions from ownership changes text file

In [140]:
find_name('39029')

[{'name': 'TESORO LOGISTICS NORTHWEST PIPELINE LLC',
  'start_year': 2013,
  'end_year': 2017,
  'id_': '39029'}]

Was sold by Chevron to Tesoro in 2013, but Tesoro set up a new unit for this pipeline, so we do not need to take any action.

In [141]:
company_groups.to_feather(f'../preprocessed_data/company_groups_{today}.feather')
m_as.to_feather(f'../preprocessed_data/m_as_{today}.feather')